In [31]:
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from numpy import sort
from sklearn.feature_selection import SelectFromModel

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from itertools import combinations

In [64]:
df = pd.read_csv("QB-Train.csv")
#df = df.drop(["year","name","college","position"],1)
df.head()
#377

,year,name,college,position,height_in,weight_lbs,40yards,vert_leap_in,broad_jump_in,shuttle,...,pct,yds,td,int,rate,heisman,bling,awards,allamerican,drafted
0,2007,zac taylor,Nebraska,QB,74.38,216,5.08,30.5,110.0,4.60,...,57.3,5853.0,45.0,20.0,130.3,0.0,1.0,1.0,0.0,N
1,2010,zac robinson,Oklahoma State,QB,74.50,214,4.68,35.0,110.0,4.40,...,61.1,8317.0,66.0,31.0,146.6,0.0,0.0,0.0,0.0,Y
2,2013,zac dysert,Miami (OH),QB,75.00,227,4.81,NaN,NaN,NaN,...,63.8,12013.0,73.0,51.0,132.4,0.0,0.0,0.0,0.0,Y
3,1997,wally richardson,Penn State,QB,76.40,221,4.95,31.0,109.0,4.47,...,54.6,4419.0,27.0,14.0,117.1,0.0,0.0,0.0,0.0,Y
4,2006,vince young,Texas,QB,76.60,229,4.48,NaN,NaN,NaN,...,61.8,6040.0,44.0,28.0,144.9,0.0,1.0,1.0,1.0,Y


In [54]:
from splinter import Browser
from sys import platform
import os
import csv
import re

executable_path = None

if platform == "linux" or platform == "linux2":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-linux")}
elif platform == "darwin":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-mac")}
elif platform == "win32":
    executable_path = {'executable_path': os.path.join("..","..","binaries", "chromedriver-windows.exe")}

browser = Browser('chrome', **executable_path, headless=False)

In [67]:
df["hasstats"] = ""

In [65]:
for index, row in df.iterrows():
    #print(df.loc[index,"hasstats"])
    if (df.loc[index,"hasstats"] != 1.0):
        if (df.loc[index,"hasstats"] != 0.0):
            x = row["name"]
            browser.visit(f"https://www.sports-reference.com/cfb/search/search.fcgi?hint=&search={x}")
            if (len(browser.find_by_css("table#passing")) > 0):
                if (len(browser.find_by_css("ul#bling")) > 0):
                    df.loc[index,"bling"] = 1
                else:
                    df.loc[index,"bling"] = 0
                if (len(browser.find_by_css("li.bling_all_star")) > 0):
                    df.loc[index,"allamerican"] = 1
                else:
                    df.loc[index,"allamerican"] = 0
                if (len(browser.find_by_css("li.bling_hof")) > 0):
                    df.loc[index,"heisman"] = 1
                else:
                    df.loc[index,"heisman"] = 0
                if (len(browser.find_by_css("div#leaderboard_awards")) > 0):
                    df.loc[index,"awards"] = 1
                else:
                    df.loc[index,"awards"] = 0

                df.loc[index, "cmp"] = browser.find_by_css("table#passing tfoot tr.bold")[0].find_by_css("td[data-stat='pass_cmp']").text
                df.loc[index, "att"] = browser.find_by_css("table#passing tfoot tr.bold")[0].find_by_css("td[data-stat='pass_att']").text
                df.loc[index, "pct"] = browser.find_by_css("table#passing tfoot tr.bold")[0].find_by_css("td[data-stat='pass_cmp_pct']").text
                df.loc[index, "yds"] = browser.find_by_css("table#passing tfoot tr.bold")[0].find_by_css("td[data-stat='pass_yds']").text
                df.loc[index, "td"] = browser.find_by_css("table#passing tfoot tr.bold")[0].find_by_css("td[data-stat='pass_td']").text
                df.loc[index, "int"] = browser.find_by_css("table#passing tfoot tr.bold")[0].find_by_css("td[data-stat='pass_int']").text
                df.loc[index, "rate"] = browser.find_by_css("table#passing tfoot tr.bold")[0].find_by_css("td[data-stat='pass_rating']").text
                df.loc[index,"hasstats"] = 1
            elif (len(browser.find_by_css("div.search-item")) == 0):
                df.loc[index,"bling"] = 0
                df.loc[index,"allamerican"] = 0
                df.loc[index,"heisman"] = 0
                df.loc[index,"awards"] = 0
                df.loc[index,"hasstats"] = 0
            else:
                listed = browser.find_by_css("div.search-item")            
                for x in listed:
                    if (re.sub(r'\(.*\)', "", row["college"]).strip() in x.find_by_css("div.search-item-league").text):
                        x.find_by_css("div.search-item-name a").click()
                        if (len(browser.find_by_css("table#passing")) > 0):
                            if (len(browser.find_by_css("ul#bling")) > 0):
                                df.loc[index,"bling"] = 1
                            else:
                                df.loc[index,"bling"] = 0
                            if (len(browser.find_by_css("li.bling_all_star")) > 0):
                                df.loc[index,"allamerican"] = 1
                            else:
                                df.loc[index,"allamerican"] = 0
                            if (len(browser.find_by_css("li.bling_hof")) > 0):
                                df.loc[index,"heisman"] = 1
                            else:
                                df.loc[index,"heisman"] = 0
                            if (len(browser.find_by_css("div#leaderboard_awards")) > 0):
                                df.loc[index,"awards"] = 1
                            else:
                                df.loc[index,"awards"] = 0

                            df.loc[index, "cmp"] = browser.find_by_css("table#passing tfoot tr.bold")[0].find_by_css("td[data-stat='pass_cmp']").text
                            df.loc[index, "att"] = browser.find_by_css("table#passing tfoot tr.bold")[0].find_by_css("td[data-stat='pass_att']").text
                            df.loc[index, "pct"] = browser.find_by_css("table#passing tfoot tr.bold")[0].find_by_css("td[data-stat='pass_cmp_pct']").text
                            df.loc[index, "yds"] = browser.find_by_css("table#passing tfoot tr.bold")[0].find_by_css("td[data-stat='pass_yds']").text
                            df.loc[index, "td"] = browser.find_by_css("table#passing tfoot tr.bold")[0].find_by_css("td[data-stat='pass_td']").text
                            df.loc[index, "int"] = browser.find_by_css("table#passing tfoot tr.bold")[0].find_by_css("td[data-stat='pass_int']").text
                            df.loc[index, "rate"] = browser.find_by_css("table#passing tfoot tr.bold")[0].find_by_css("td[data-stat='pass_rating']").text
                            df.loc[index,"hasstats"] = 1
                            break
                        else:
                            df.loc[index,"bling"] = 0
                            df.loc[index,"allamerican"] = 0
                            df.loc[index,"heisman"] = 0
                            df.loc[index,"awards"] = 0
                            df.loc[index,"hasstats"] = 0
                            break
                    else:
                        df.loc[index,"bling"] = 0
                        df.loc[index,"allamerican"] = 0
                        df.loc[index,"heisman"] = 0
                        df.loc[index,"awards"] = 0
                        df.loc[index,"hasstats"] = 0

In [68]:
df.to_csv("QB-Train.csv", index=False)